In [ ]:
import torch
import pandas as pd
import numpy as np
import datasets
from load_models_and_data import load_vocabulary, load_embeddings, text_to_embeddings, calc_cosine_sim, calculate_similarities
from tqdm import tqdm
tqdm.pandas()
from TwoTowerNN import QryTower, DocTower, TripletEmbeddingDataset, run_hyperparameter_tuning
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import Dataset, DataLoader,  SubsetRandomSampler
from sklearn.model_selection import KFold, train_test_split
import os
import wandb
from dotenv import load_dotenv

API key loaded successfully


wandb: Currently logged in as: nnamdi-odozi (nnamdi-odozi-ave-actuaries) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from datasets import load_dataset

# Loading datasets from Hugging Face
ds_soft_neg = load_dataset("cocoritzy/week_2_triplet_dataset_soft_negatives")
ds_hard_neg = load_dataset("cocoritzy/week_2_triplet_dataset_hard_negatives")


In [ ]:
# Paths to your files
embeddings_path = "./downloaded_model/embeddings.pt"
vocab_path = "./downloaded_model/tkn_ids_to_words.csv"

# Load embeddings and vocabulary
print("Loading embeddings and vocabulary...")
embeddings = load_embeddings(embeddings_path)
word_to_idx = load_vocabulary(vocab_path)

print(f"Loaded embeddings with shape: {embeddings.shape}")
print(f"Loaded vocabulary with {len(word_to_idx)} tokens")

# Example usage (uncomment when ready to test)
sample_text = "This is a test sentence"
embeddings_result = text_to_embeddings(sample_text, word_to_idx, embeddings)
print(f"Embedded text shape: {embeddings_result.shape}")

# Testing - Set numpy print options
np.set_printoptions(precision=4, suppress=True, threshold=10)  # threshold limits number of elements shown
numpy_array = embeddings_result.detach().numpy()
print("Embedding array with custom formatting:")
print(numpy_array)


In [ ]:
ds_soft_neg

In [ ]:
df_soft_neg  = pd.DataFrame(ds_soft_neg['train'])
df_hard_neg  = pd.DataFrame(ds_hard_neg['train'])

In [ ]:
embedded_query = text_to_embeddings(df_soft_neg['query'][0], word_to_idx, embeddings)
embedded_positive = text_to_embeddings(df_soft_neg['positive_passage'][0], word_to_idx, embeddings)
embedded_negative = text_to_embeddings(df_soft_neg['negative_passage'][0], word_to_idx, embeddings)

embedded_query.shape

In [ ]:
a = embedded_query.mean(dim=0)
b = embedded_positive.mean(dim=0)
c = embedded_negative.mean(dim=0)
a.shape


In [ ]:
import torch.nn.functional as F

cosine_similarity = F.cosine_similarity(a, c, dim=0)
print(f"Cosine similarity between query and positive passage: {cosine_similarity.item()}")

In [ ]:

# # Process the dataframe using apply just for first five rows
# print("Calculating similarities... This may take a while depending on dataframe size.")
# similarities = df_soft_neg[0:5].progress_apply(
#     lambda row: calculate_similarities(row, word_to_idx, embeddings), 
#     axis=1
# )

# # Join the similarities to the dataframe
# df_soft_neg_ext = pd.concat([df_soft_neg[0:5], similarities], axis=1)

# # Show a sample of the results
# #print(df_soft_neg_ext[['query_pos_sim', 'query_neg_sim', 'pos_neg_sim']].head())
#print(df_soft_neg_ext.head())
#print(df_soft_neg_ext.columns)

In [ ]:

# Process the dataframe using apply
print("Calculating similarities... This may take a while depending on dataframe size.")
similarities = df_soft_neg.progress_apply(
    lambda row: calculate_similarities(row, word_to_idx, embeddings), 
    axis=1
)

# Join the similarities to the dataframe
df_soft_neg_ext = pd.concat([df_soft_neg, similarities], axis=1)
print(df_soft_neg_ext.head())
# Show a sample of the results
#print(df_soft_neg_ext[['query_pos_sim', 'query_neg_sim', 'pos_neg_sim']].head())

#print(df_soft_neg_ext[['query_pos_sim', 'query_neg_sim', 'pos_neg_sim']].mean())

# Calculate how often the positive passage is ranked higher than negative
#higher_count = (df_soft_neg_ext['query_pos_sim'] > df_soft_neg_ext['query_neg_sim']).sum()
#total = len(df_soft_neg_ext)
#print(f"\nPositive passage ranked higher than negative: {higher_count} out of {total} ({higher_count/total:.2%})")



In [ ]:
# Process the dataframe using apply
print("Calculating similarities... This may take a while depending on dataframe size.")
similarities = df_hard_neg.progress_apply(
    lambda row: calculate_similarities(row, word_to_idx, embeddings), 
    axis=1
)

# Join the similarities to the dataframe
df_hard_neg_ext = pd.concat([df_hard_neg, similarities], axis=1)
print(df_hard_neg_ext.head())
# Show a sample of the results
#print(df_hard_neg_ext[['query_pos_sim', 'query_neg_sim', 'pos_neg_sim']].head())

#print(df_hard_neg_ext[['query_pos_sim', 'query_neg_sim', 'pos_neg_sim']].mean())

# Calculate how often the positive passage is ranked higher than negative
#higher_count = (df_hard_neg_ext['query_pos_sim'] > df_hard_neg_ext['query_neg_sim']).sum()
#total = len(df_hard_neg_ext)
#print(f"\nPositive passage ranked higher than negative: {higher_count} out of {total} ({higher_count/total:.2%})")



In [ ]:
df_all_neg_ext = pd.concat([df_soft_neg_ext, df_hard_neg_ext])
df_all_neg_ext.head()

In [ ]:
# Save DataFrames to pickle format
df_soft_neg_ext.to_pickle("data/df_soft_neg_ext.pkl")
df_hard_neg_ext.to_pickle("data/df_hard_neg_ext.pkl")
df_all_neg_ext.to_pickle("data/df_all_neg_ext.pkl")

In [4]:
# Function to load a DataFrame from pickle if the file exists
def load_df_if_exists(file_path):
    if os.path.exists(file_path):
        return pd.read_pickle(file_path)
    else:
        print(f"File not found: {file_path}")
        return None

# Load DataFrames
df_soft_neg_ext = load_df_if_exists("data/df_soft_neg_ext.pkl")
df_hard_neg_ext = load_df_if_exists("data/df_hard_neg_ext.pkl")
df_all_neg_ext = load_df_if_exists("data/df_all_neg_ext.pkl")


In [8]:
df_soft_neg_ext.head()

,query_id,query,positive_passage,negative_passage,negative_from_query_id,avg_query_embedding,avg_pos_embedding,avg_neg_embedding
0,19699,what is rba,Results-Based Accountability® (also known as R...,I finally found some real salary data for phys...,86595,"[0.6579812, 0.24213153, 0.057250064, -0.825741...","[0.39086032, 0.3319433, 0.1275278, -0.80645, 0...","[0.569893, 0.18935415, 0.1920344, -0.7171183, ..."
1,19700,was ronald reagan a democrat,"From Wikipedia, the free encyclopedia. A Reaga...",The Pacific Ocean lies to the east while the S...,66360,"[-0.6998242, -0.24631366, -0.20571017, 0.24202...","[0.27046937, 0.2619914, 0.049588773, -0.618945...","[0.17404862, 0.21760696, -0.10469024, -0.23737..."
2,19701,how long do you need for sydney and surroundin...,Sydney is the capital city of the Australian s...,"Probiotics are found in foods such as yogurt, ...",88507,"[0.16817716, 0.29739928, -0.36492547, 0.064426...","[0.39110944, 0.23566554, 0.063871, -0.36585316...","[0.61134595, 0.36615297, 0.28972, -0.6924668, ..."
3,19702,price to install tile in shower,1 Install ceramic tile floor to match shower-A...,Iodine is critical to thyroid health and funct...,87550,"[-0.06541735, 0.2755244, 0.19580394, 0.4023429...","[0.69151133, 0.5770993, 0.22074986, -0.7754023...","[0.3590759, -0.036869664, 0.17250647, -0.53339..."
4,19703,why conversion observed in body,Conversion disorder is a type of somatoform di...,The answer to the question how much does it co...,61479,"[-0.13369766, -0.30740747, 0.5450557, 0.391294...","[0.42539537, 0.13814452, 0.37000972, -0.632320...","[0.5729694, 0.314426, 0.13929352, -0.9086552, ..."


### Twin Tower Network

In [ ]:
# Create tower instances
qryTower = QryTower()
docTower = DocTower()


# Define hyperparameters
batch_size = 128
num_epochs = 1 # adjust num of epochs here
dataset_size = len(df_soft_neg_ext)  # or len(df_hard_neg_ext) depending on the dataset you want to use
steps_per_epoch = dataset_size // batch_size
total_steps = steps_per_epoch * num_epochs
learning_rate = 1e-3
embedding_dim = 128 
margin = 0.2 

In [ ]:
# Create the dataset
dataset = TripletEmbeddingDataset(df_soft_neg_ext)

In [ ]:
dataloader = DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=True,
    #num_workers=2,  # Adjust based on your machine's capabilities
    pin_memory=True  # Set to True if using GPU
)

In [ ]:
qry = torch.randn(batch_size, embedding_dim)  # Query embeddings
pos = torch.randn(batch_size, embedding_dim)  # Positive doc embeddings
neg = torch.randn(batch_size, embedding_dim)  # Negative doc embeddings

#qry = df1['q']


# Set up the AdamW optimizer
optimizer = torch.optim.AdamW([
    {'params': qryTower.parameters()},
    {'params': docTower.parameters()}
], lr=learning_rate)

# Add learning rate scheduler (ReduceLROnPlateau)
scheduler = ReduceLROnPlateau(
    optimizer,
    mode='min',       # Reduce LR when monitored value stops decreasing
    factor=0.5,       # Multiply LR by this factor when reducing
    patience=2,       # Number of epochs with no improvement after which LR will be reduced
    verbose=True      # Print message when LR is reduced
)



In [ ]:
# Training loop (simplified example)
for epoch in range(num_epochs):
    qryTower.train()
    docTower.train()
    
    
    total_loss = 0
    for batch in dataloader:
        # Get embeddings from batch
        query_emb = batch['query']
        pos_emb = batch['positive']
        neg_emb = batch['negative']
        
        # Forward pass through towers
        query_encoded = qryTower(query_emb)
        pos_encoded = docTower(pos_emb)
        neg_encoded = docTower(neg_emb)
        
        # Calculate similarities
        pos_sim = torch.nn.functional.cosine_similarity(query_encoded, pos_encoded)
        neg_sim = torch.nn.functional.cosine_similarity(query_encoded, neg_encoded)
        
        # Triplet loss
        margin = margin
        loss = torch.clamp(margin - pos_sim + neg_sim, min=0).mean()
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item() * len(query_emb)
    
    # Calculate average loss
    avg_loss = total_loss / len(dataset)
    
    # Update scheduler
    scheduler.step(avg_loss)
    
    # Print epoch results
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}, "
          f"LR: {optimizer.param_groups[0]['lr']:.6f}")

In [13]:
# Run the hyperparameter tuning with your dataframe
best_params, final_qry_tower, final_doc_tower = run_hyperparameter_tuning(
    df_all_neg_ext,
    output_dims=[128],
    batch_sizes=[256, 512],
    n_folds=2,
    epochs=20
)

# Print the best parameters found
print(f"Best output dimension: {best_params['output_dim']}")
print(f"Best batch size: {best_params['batch_size']}")
print(f"Best validation loss: {best_params['avg_cv_loss']:.4f}")

/root/MLX_Week2/mlx_env/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(




--------------------------------------------------
Training with output_dim=128, batch_size=256
--------------------------------------------------

Fold 1/2


Epoch 1/20 (Val): 100%|██████████| 250/250 [00:07<00:00, 31.58it/s]


Epoch 1/20, Train Loss: 0.0618, Val Loss: 0.0561, LR: 0.001000
New best model saved with validation loss: 0.0561


Epoch 2/20 (Val): 100%|██████████| 250/250 [00:07<00:00, 31.36it/s]


...

Epoch 19/20, Train Loss: 0.0528, Val Loss: 0.1004, LR: 0.000125


Epoch 20/20 (Val): 100%|██████████| 63/63 [00:03<00:00, 18.80it/s]

Epoch 20/20, Train Loss: 0.0524, Val Loss: 0.1003, LR: 0.000063


avg_cv_loss,█▁
batch_size,▁█
epoch,▁▁▃▄▅▇▇▇▁▂▄▅▅▆▇▂▂▂▃▅▅▆▆▇█▁▁▂▂▃▄▅▅▇▇█▁▁▃▅
learning_rate,█████▃▂▂█████▄▄▃▂▁▁▁██▄▃▁█████▂▁▁██▄▄▄▃▂
output_dim,▁▁
train_loss,▄▃▃▁▁▁▄▃▃▃▂▂▂▁▁▁▁▃▂▂▁▅▄▃▃▂▂▁▁▁▁▁█▇▇▅▅▄▄▄
val_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▇▇▇▇▇
avg_cv_loss,0.0511
batch_size,512
epoch,20
learning_rate,6e-05


Best output dimension: 128
Best batch size: 512
Best validation loss: 0.0511


In [11]:
# Code to upload final model to wandb
import wandb
import os
import time
from dotenv import load_dotenv

# Load your API key from config.txt
def load_api_key_from_config(config_path="config.txt"):
    try:
        with open(config_path, "r") as f:
            first_line = f.readline().strip()
            if "=" in first_line:
                api_key = first_line.split("=")[1].strip()
            else:
                api_key = first_line
        return api_key
    except FileNotFoundError:
        print(f"Config file not found at {config_path}")
        return None

# Set up wandb - only set API key if wandb.run doesn't exist yet
if wandb.run is None:
    api_key = load_api_key_from_config()
    if api_key:
        os.environ["WANDB_API_KEY"] = api_key
        wandb.login()
        print("Successfully logged in to Weights & Biases")
    else:
        print("Failed to load API key")

# Check if there's an active run, only initialize if needed
if wandb.run is None:
    run = wandb.init(
        project="twin-tower-model",
        name="final-model-summary",  # Changed name to indicate this is a summary
        config={
            "output_dim": best_params["output_dim"],
            "batch_size": best_params["batch_size"],
            "architecture": "Twin Tower Network",
            "dataset": "MS MARCO"
        }
    )
else:
    run = wandb.run
    # Update the run with additional metadata if needed
    run.config.update({
        "architecture": "Twin Tower Network",
        "dataset": "MS MARCO"
    })

# Upload the model with a timestamp to avoid conflicts
timestamp = int(time.time())
artifact_name = f"twin-tower-final-model-{timestamp}"

model_artifact = wandb.Artifact(
    name=artifact_name, 
    type="model",
    description="Twin Tower model trained on full training data with optimal hyperparameters"
)

final_model_path = "checkpoints/final_model/final_model.pt"
model_artifact.add_file(final_model_path)
wandb.log_artifact(model_artifact)

print(f"Final model uploaded to Weights & Biases project: {run.project}")

# Only finish the run if we created it in this script
if run.name == "final-model-summary":
    wandb.finish()

Final model uploaded to Weights & Biases project: twin-tower-model
